<a href="https://colab.research.google.com/github/mikio-bonjour/Python_code/blob/%E5%BE%AE%E5%88%86%E7%A9%8D%E5%88%86/%E5%8D%98%E7%B4%94%E3%83%8F%E3%82%9A%E3%83%BC%E3%82%BB%E3%83%95%E3%82%9A%E3%83%88%E3%83%AD%E3%83%B3%E3%81%AE%E5%AE%9F%E8%A3%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単純パーセプトロンを 3 通りで実装
- **パターン A** : 単純に Python (`math`)で計算
- **パターン B** : NumPy でベクトル化
- **パターン C** : PyTorch で層を宣言  
それぞれで **ŷ ≈ 0.8808** の出力結果が得られるか確認します。

## パターン A — 単純に Python で計算
- **重み付き和 → シグモイド** をそのまま書く
- 手計算との 1:1 対応を確認するのが目的

In [1]:
import math

def sigmoid(z: float) -> float:
    """
  標準ロジスティックシグモイド活性化関数の定義
    """
    return 1 / (1 + math.exp(-z))

def perceptron(inputs, weights, activation=sigmoid):
    """
    単一ニューロン（パーセプトロン）の出力を計算する。
      inputs  : x0, x1, …  （バイアス項を含む）
      weights : w0, w1, …
      activation : 活性化関数（デフォルト: シグモイド）
    戻り値は (線形結合 z, 出力 \hat{y})
    """
    z = sum(i * w for i, w in zip(inputs, weights))
    return z, activation(z)

x = [1.0, 2.0, -1.0]     # x0=1 (バイアス), x1=2, x2=-1
w = [-1.0, 2.0, 1.0]     # w0=-1, w1=2, w2=1

z_val, y_hat = perceptron(x, w)

print(f"z = {z_val}")        # -> 2.0
print(f"ŷ = {y_hat:.6f}")    # -> 0.880797


z = 2.0
ŷ = 0.880797


## パターン B — NumPy でベクトル／バッチ対応
- 行列積 `@` が **線形層** と同じ働きになる
- 「バッチ（複数サンプル）」を一括処理する例で示す


In [ ]:
import numpy as np

x = np.array([1.0, 2.0, -1.0])
w = np.array([-1.0, 2.0, 1.0])

z = x @ w
y = 1 / (1 + np.exp(-z))

# ─ バッチ計算例 ───────────────────
X_batch = np.array([
    [1.0, 2.0, -1.0],  # サンプル1
    [0.5, 1.5, 2.0],   # サンプル2
    [3.0, -1.0, 0.0]   # サンプル3
])
Z = X_batch @ w
Y = 1 / (1 + np.exp(-Z))
print(Y)

## パターン C — PyTorch で層を宣言
- `nn.Linear` が **w·x + b**, `nn.Sigmoid` が活性化
- Colab は PyTorch がプリインストール済み
- 乱数初期化を避けるため **手動で重み・バイアスを設定* する

In [ ]:
import torch
import torch.nn as nn

# 手動で設定するパラメータ（重み w1, w2 と バイアス w0）
w0, w1, w2 = -1.0, 2.0, 1.0

# 入力データ x: 2次元特徴量を持つサンプルが1つだけある (batch=1, features=2)
# ここではバイアス項は含まず、単純に2次元の実数値のみ
x = torch.tensor([[2.0, -1.0]])  # shape: (1, 2)

# nn.Sequential:
# - まず nn.Linear(2, 1, bias=True) で入力2次元、出力1次元の線形変換を定義
# - 続いて nn.Sigmoid() で活性化関数としてシグモイドを適用
model = nn.Sequential(
    nn.Linear(2, 1, bias=True),
    nn.Sigmoid()
)

# with torch.no_grad(): を使うことで、以下の操作で
# PyTorch が勾配を追跡しないようにする（トレーニング目的ではないので不要）
with torch.no_grad():
    # model[0] は nn.Sequential(...) の一番目の層 (nn.Linear(2, 1, bias=True))
    # weight.copy_() で、重みの値を手動で [ [w1, w2] ] にコピー
    # shape を (1,2) として対応させる
    model[0].weight.copy_(torch.tensor([[w1, w2]]))

    # bias.copy_() で、バイアスの値を手動で [w0] にコピー
    # shape を (1,) として対応させる
    model[0].bias.copy_(torch.tensor([w0]))

# 設定したパラメータでモデルを実行する
y_hat = model(x)

# テンソル（単一スカラー）の値を Python の float として取得して表示
print(y_hat.item())